<img src="uva_seal.png">  

## Spark SQL and DataFrames

### University of Virginia
### DS 7200: Distributed Computing
### Last Updated: September 6, 2023

---  

### Sources 

Learning Spark, Chapter 9: Spark SQL

https://spark.apache.org/docs/latest/sql-programming-guide.html

https://www.datacamp.com/community/tutorials/apache-spark-tutorial-machine-learning

Demonstration of several useful DataFrame operations:  
https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html

### OBJECTIVES
- Introduction to Spark SQL, the interface for working with structured and semistructured data
- Introduce DataFrames and show basic functionality
- Explain the benefits of the Parquet format

### CONCEPTS AND FUNCTIONS
- Schema
- SQL
- Dataset and DataFrame
- Partition
- Parquet files

---  

### 1. Overview

Two important ways of working with big data in Spark: 

- through Spark SQL
- using DataFrames

They also interoperate

### 2. Data Schema in Spark

The schema in Spark defines the data structure. For each field, a 3-tuple is specified: `(column name, data type, nullable)`  

---  

**Example of schema with two Fields *author* and *pages*, which cannot contain null values**
```
schema = StructType([StructField("author", StringType(), False), StructField("pages", IntegerType(), False)])
```
---  

It is possible to allow Spark to infer the schema of your data, but it's preferable to feed it the schema:

- avoids having Spark launch a separate job to read a large fraction of the data to infer schema
- early detection of errors if the data doesn't match the schema
- Spark inference may be incorrect. For example, it may think all numerical data are strings.

### This schema is different from database schema

A database *schema* is the structure that represents the logical view of the entire database.  
It defines how data is organized and how relations among them are associated.  
This is implemented through the use of tables, views, and integrity constraints.

 <img src="./db_schema.png" width=500> 

### 3. Common Spark Data Types

- integer types, all `int` in python:
  - ShortType
  - IntegerType
  - LongType
  - FloatType
  - DoubleType
- StringType
- BooleanType

### 4. SQL in Ten Seconds (tongue in cheek)


SQL is a structured query language used to communicate with relational databases.  
Commands include CREATE, SELECT, UPDATE, ALTER, INSERT INTO, DROP, DELETE.  
This course will focus on SELECT.

### 5. Spark SQL Capabilities:

- load data from various structured formats including JSON, Hive, Parquet  
- query data using SQL inside Spark or from external tools that connect to Spark (e.g., `Tableau`) 
- Spark SQL integrates between SQL and Python/Java/Scala/R code. Can do things like join RDDs and SQL tables.

### 6. Note on Spark SQL Development

 Spark SQL has been heavy development area in new releases.  
 As the module involves massive amounts of data, optimizing operations is valuable.  
 We will discuss the optimizer behind Spark SQL.
 
---

### 7. Dataset and DataFrame

- A Dataset is a distributed collection of data   
- A Dataset can be constructed from JVM objects and then manipulated using functional transformations (`map()`, `flatMap()`, `filter()`, etc.)  
- A DataFrame is a Dataset organized into named columns   

In practice, you will be thinking in terms of `DataFrames`, and not `Datasets`.  For users familiar with dataframes from R and Python, they are similar, yet with operations distinct to Spark.  As an example, adding a new column to a DataFrame is executed using `withColumn()`.  This may feel more formal compared to R and Python.  

Additionally - when compared to R and Python - **the Spark DataFrame uses richer optimizations under the hood.  
The structure makes use of distributed computing, in the same manner as RDDs.**  

DataFrames can be constructed from a wide array of sources such as: structured data files, tables in Hive, external databases, or existing RDDs.  

The DataFrame API is available in Scala, Java, Python, and R. 

### 8. DataFrames vs RDDs  

Now that we have two powerful objects that parallelize data, we have more flexibility, but this can lead to confusion.  When is it better to use DataFrames, and when is it better to use RDDs?  

Here are some recommendations:   

- In general, most work can be done with DataFrames  

- Use DataFrames to use high-level expressions, to perform SQL queries to explore the data, and to gain columnar access.  For example, if you are thinking about the data by field names, you probably want the data in a DataFrame.

- For machine learning and building predictive models, DataFrames are recommended. You will be exploring the data by column, and building features from the columns of data.  
- RDDs can be useful to perform low-level transformations and actions on unstructured data. For example, filtering strings and performing other simple transformations on text is best done with RDDs.  In these cases, the analyst doesn't care about field names, and there is no need to impose schema on the data.  

- Use RDDs when you want to manipulate the data with functional programming constructs rather than domain specific expressions.

### 9. Creating a DataFrame

There are multiple ways to do this:
- use a function such as `read.csv()` to read data from files into DataFrames (most common)
- pass data to `createDataFrame()` with schema
- conversion from RDD using `toDF()`

**Example 1: Create DataFrame from RDD using `toDF()`**

---  
```
# import modules 
from pyspark.sql import Row

# Map the RDD to a DF

df = rdd.map(lambda line: Row(longitude=line[0], 
                              latitude=line[1], 
                              housingMedianAge=line[2],
                              totalRooms=line[3],
                              totalBedRooms=line[4],
                              population=line[5], 
                              households=line[6],
                              medianIncome=line[7],
                              medianHouseValue=line[8])).toDF()
```
---  

**Example 2: Create DataFrame by passing data and schema to `createDataFrame()`**

In [ ]:
# import context manager: SparkSession
from pyspark.sql import SparkSession

# import data types
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# set up the session
spark = SparkSession.builder.getOrCreate()

# create some data; list of tuples
data = [
    (0, "ChatGPT is all the rage"),
    (1, "Google released BARD to compete"),
    (2, "What does AWS think about this?")
]

# define schema; each field holds (name, data type, nullable)
# for large number of fields, best to automate schema construction
schema = StructType([StructField('id', IntegerType(), False), 
                     StructField('sentence', StringType(), False)])

# create df by passing data, schema
sentenceDataFrame = spark.createDataFrame(data, schema)

# print first few records
sentenceDataFrame.show(3, False)

# print data type
print(type(sentenceDataFrame))

**Example 3: Create a DataFrame from some JSON data with spark.read()**  
(For an example of JSON data see: http://json.org/example.html)


In [ ]:
import os
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

# Read data in json format
df = spark.read.json("people.json")

# Displays the content of the DataFrame to stdout
df.show()

**Creating an RDD from a DataFrame**

This is very simple: `df.rdd`

Here we convert our df containing sentences:

In [ ]:
sentence_rdd = sentenceDataFrame.rdd
print(sentence_rdd.take(2))
print(type(sentence_rdd))

### 10. Some Useful DataFrames Operations

Next we explore subsetting, filtering, and aggregation among others.

In [ ]:
# Read data in json format
df = spark.read.json("people.json")
df.show()

### Selecting

There are three ways to select columns and we show them all. Find your favorite.

- using bracket operator
- using `col()` method
- using dot operator (my favorite)

We see them below

### Filtering

Keep records where age > 21

In [ ]:
# bracket operator
df.filter(df['age'] > 21).show()

In [ ]:
# dot operator
df.filter(df.age > 21).show()

In [ ]:
# column operator
from pyspark.sql.functions import col

df.filter(col('age') > 21).show()

Keep records subject to filters on name, then sort

In [ ]:
from pyspark.sql.functions import asc

# alternatively using df.name instead of col("name")
df.filter((df.name == "Andy") | (df.name == "Michael")).sort(asc("name")).show()

Fetch records with age *null*

In [ ]:
df.filter(col("age").isNull()).show() 

Fetch records with age *not null*

In [ ]:
df.filter(col("age").isNotNull()).show() 

### where() is equivalent to filter()

In [ ]:
df.where((df.name == "Andy") | (df.name == "Michael")).sort(asc("name")).show()

### Impute missing with 0 (just for illustration; not a great idea for this data)

In [ ]:
df.fillna(0).show()

### Summarize the age field

In [ ]:
df.describe("age").show()

### 11. Spark SQL Queries and Temp Views

Spark SQL is at the interface of SQL and DataFrames.    
To write SQL queries against DataFrames, first register DF as a `SQL temp view`, and then write the query.

A temp view is session scoped:  
- visible only to the session that created it 
- dropped when the session ends (not persisted)

**Example of SQL Query against DataFrame**

In [ ]:
# register DataFrame as temp view with name "people"
df.createOrReplaceTempView("people")

# query the view
sqlDF = spark.sql("SELECT * FROM people where name == 'Andy'")
sqlDF.show()

### 12. Aggregate on columns

SQL functions can be loaded from this library: `pyspark.sql.functions`

We will load some stock data to demonstrate aggregation.

In [ ]:
# import data types and functions
from pyspark.sql.types import StructType, StructField, DateType, StringType, LongType, FloatType
from pyspark.sql import functions as F

# set up schema
stock_schema = StructType([StructField('date',DateType(),False),
                           StructField('ticker',StringType(),False),
                           StructField('close',FloatType(),False),
                           StructField('adjusted_close',FloatType(),False),
                           StructField('volume',LongType(),False),
                         ])

In [ ]:
# Read in stock data. Source: Yahoo! finance
DATAPATH_STOCKS = './amzn_msft_prices.csv'

df_stx = spark.read.csv(DATAPATH_STOCKS, header=True, schema=stock_schema)
df_stx.show()

In [ ]:
# check the schema

df_stx.printSchema()

We will calculate some statistics on each stock:
- minimum closing price
- maximum closing price
- minimum volume
- maximum volume

In [ ]:
agg_df = df_stx.groupBy("ticker").agg(F.min("close"), F.max("close"), F.min("volume"), F.max("volume"))
agg_df.show()

**IMPORTANT NOTE**  
Do NOT use loops to aggregate data. Loops are run sequentially and do not harness parallelization.  
Using the `groupBy()` method will do the job using parallelization.

### 13. Input/Output

#### 13A. I/O with DataFrames

Here we show various examples of reading and writing dataframes.  
This code is for illustration only.

**Read in Data and Infer the Schema**

```
adult_df = spark.read.\
    format("com.spark.csv").\
    option("header", "false").\
    option("inferSchema", "true").load("dbfs:/databricks-datasets/adult/adult.data")
```

**Read and Write using Generic Functions**

```
df = spark.read.load("examples/src/main/resources/users.parquet")
df.select("name", "favorite_color").write.save("namesAndFavColors.parquet")
```

**Read and Write using Manually Specified Formats**

```
df = spark.read.load("examples/src/main/resources/people.json", format="json")
df.select("name", "age").write.save("namesAndAges.parquet", format="parquet")
```

#### 13B. Parquet Files

- Parquet is a **columnar format** that is supported by many other data processing systems  

- Stores metadata about the columns, which can provide efficiency

- Files hold binary data

- Read and writing parquet files can be MUCH faster in Spark

- Especially useful when querying columns for analytics and ML (don't generally need entire rows of data)

- Good compression and encoding options:  
  Implements a hybrid of bit packing and RLE - encoding switches based on which produces the best compression results

  *Bit Packing* - Usually 32 or 64 bits of storage dedicated per integer.  
  For small integers, packs multiple integers into same space for efficient storage.  
  
  *Run Length Encoding* - for sequence of duplicate values, store single value with number of occurrences.


<img src="./RLE.png">

#### 13C. Partition Discovery

Database tables can be partitioned to make querying more efficient.  
For example, the data can be
split by gender and country, producing smaller tables.  
If the analyst is only interested in a single country, the query will run faster.


In a partitioned table, data are usually stored in different directories, with partitioning column values encoded in the path of each partition directory.  

All built-in file sources (including Text/CSV/JSON/ORC/Parquet) are able to discover and infer partitioning information automatically. 

In [ ]:

path
└── to
    └── table
        ├── gender=male
        │   ├── ...
        │   │
        │   ├── country=US
        │   │   └── data.parquet
        │   ├── country=CN
        │   │   └── data.parquet
        │   └── ...
        └── gender=female
            ├── ...
            │
            ├── country=US
            │   └── data.parquet
            ├── country=CN
            │   └── data.parquet
            └── ...


**Examples of writing DF to Parquet file, partitioning columns**

```
df = df.withColumn('end_month', F.month('end_date'))
df = df.withColumn('end_year', F.year('end_date'))
df.write.partitionBy("end_year", "end_month").parquet("/tmp/sample_table")
```

**TRY FOR YOURSELF (UNGRADED EXERCISES)**

1) Given the stock dataframe, use Spark SQL to select all AMZN records. 

In [ ]:
# SOLUTION

# register stock dataFrame as temp view with name "stocks"
df_stx.createOrReplaceTempView("stocks")

# query the view
sqlDF = spark.sql("SELECT * FROM stocks where ticker == 'AMZN'")
sqlDF.show()

2) Given the stock dataframe, do an aggregation to compute minimum, mean, and maximum adjusted close for each stock.

In [ ]:
df_stx.groupBy("ticker").agg(F.min("adjusted_close"), F.mean("adjusted_close"), F.max("adjusted_close")).show()

3) Select the date, ticker, and adjusted_close columns, saving this data as a parquet file.

In [ ]:
df_stx.select("date","ticker","adjusted_close").write.save("stocks.parquet", format="parquet")

4) Load the parquet file into a new dataframe and verify that things look correct.

In [ ]:
test = spark.read.load("stocks.parquet")
test.show()

### 14. Summary

You should now have a basic understanding of Spark SQL, DataFrames, and how to use some of the common transformations on DataFrames.  
With practice, you will gain comfort in selecting and processing data with Spark SQL and DataFrames, which is essential.

Additionally, you should have some sense of when DataFrames are preferred over RDDs, and vice versa.
